In [0]:
import pandas as pd
import numpy as np
import json

In [2]:
file = "combined_{}.csv"
dfs = []
for i in range(3):
    fp = file.format(i+1)
    read = pd.read_csv(fp)
    read = read[['label', 'clean_text']]
    dfs.append(read)

dfs[2] = dfs[2][:-13000]

data = pd.concat(dfs)
data.tail()
data.reset_index(inplace=True, drop=True)
print(data.shape)
print(data[data.label == 1].shape[0], "Real")
print(data[data.label == 0].shape[0], "Fake")

data.dropna(inplace=True)
data.head(10)

(59818, 2)
31514 Real
28304 Fake


,label,clean_text
0,0,house dem aide didnt even see comeys letter ja...
1,1,ever get feeling life circles roundabout rathe...
2,0,truth might get fired october tension intell...
3,0,videos civilians killed single us airstrike i...
4,0,print iranian woman sentenced six years prison...
5,1,trying times jackie mason voice reason weeks e...
6,0,ever wonder britains iconic pop pianist gets l...
7,1,paris france chose idealistic traditional cand...
8,1,donald trump scheduled make highly anticipated...
9,1,week michael flynn resigned national security ...


# Experiment Number 1
## Naive Bayes, Real Human News and Fake Human News

Class: Label

Real: 1

Fake: 0

In [0]:
import nltk
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [4]:
combined_text = data['clean_text'].values
labels = data['label'].values
data.groupby('label').agg('count')

,clean_text
label,
0,28010
1,31449


In [0]:
tfidf = TfidfVectorizer(ngram_range=(1,2), max_df= 0.85, min_df= 0.01)

In [0]:
combined_text = tfidf.fit_transform(combined_text)

In [7]:
print("tf idf vectorized text shape: {}".format(combined_text.shape))
print("original text shape: {}".format(data['clean_text'].shape))


tf idf vectorized text shape: (59459, 4252)
original text shape: (59459,)


In [0]:
train_text, test_text, train_labels, test_labels  = train_test_split(combined_text, labels, test_size=0.2)


In [9]:
import collections

print("size of train_text: {}".format(train_text.shape))

print("size of train_labels: {}".format(train_labels.shape))
collections.Counter(train_labels)


size of train_text: (47567, 4252)
size of train_labels: (47567,)


Counter({0: 22430, 1: 25137})

In [10]:
print("size of test_text: {}".format(test_text.shape))

print("size of test_labels: {}".format(test_labels.shape))
collections.Counter(test_labels)



size of test_text: (11892, 4252)
size of test_labels: (11892,)


Counter({0: 5580, 1: 6312})

In [0]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()


In [12]:
nb.fit(train_text,train_labels)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
train_pred = nb.predict(train_text)

In [14]:
print('Naive Bayes In Sample F1 and Accuracy Scores:')
print('F1 score {:.4}%'.format(f1_score(train_labels, train_pred, average='macro')*100 ))
print ('Accuracy score {:.4}%'.format(accuracy_score(train_labels, train_pred)*100))

Naive Bayes In Sample F1 and Accuracy Scores:
F1 score 79.98%
Accuracy score 80.31%


In [15]:
data.loc[np.where(train_labels != train_pred)[0].tolist()].groupby('label').agg('count')

KeyError: ignored

In [16]:
test_pred = nb.predict(test_text)
print('Naive Bayes Out of Sample F1 and Accuracy Scores:')
print('F1 score {:.4}%'.format(f1_score(test_labels, test_pred, average='macro')*100 ))
print ('Accuracy score {:.4}%'.format(accuracy_score(test_labels, test_pred)*100))

Naive Bayes Out of Sample F1 and Accuracy Scores:
F1 score 78.67%
Accuracy score 79.05%


In [17]:
data.loc[np.where(test_labels != test_pred)[0].tolist()].groupby('label').agg('count')

KeyError: ignored

# Experiment 2
## Augmenting with Neural Fake News

905 Real News, 905 Fake (neural and human combined)

80% Train = 1448
20% Test = 362 (1/2 Real, 1/2 Human Fake)

Train on (1/2 real news, 1/2 fake news (1/4 human fake + 1/4 neural fake))

Human Fake = 543 (362 for train, 181 for test)

Neural Fake = 362 for train

In [0]:
print(df_grover.shape)
print(df_real.shape)
print(df_fake.shape)

In [0]:
trim_size = int(df_real.shape[0] * 0.6)
chosen_idx = np.random.choice(df_fake.shape[0], replace=False, size=trim_size)
df_fake_trim_2 = df_fake.iloc[chosen_idx]
df_fake_trim_2['label'] = 0
print(df_fake_trim_2.shape)
df_fake_trim_2.head()

In [0]:
trim_size = int(df_real.shape[0]*0.4)
chosen_idx = np.random.choice(df_grover.shape[0], replace=False, size=trim_size)
df_grover_trim = df_grover.iloc[chosen_idx]
df_grover_trim['label'] = 0
print(df_grover_trim.shape)
df_grover_trim.head()

In [0]:
print(df_real.shape)
df_real.head()

In [0]:
#Create Test Set first from trimmed
trim_size = int(df_real.shape[0]*.2)
mask =  np.random.choice(df_real.shape[0], trim_size, replace=False) 
neg_mask = [i for i in range(0, df_real.shape[0]) if i not in mask]
df_real_test = df_real.iloc[mask]
df_real_train = df_real.iloc[neg_mask]
print(df_real_test.shape)
print(df_real_train.shape)

In [0]:
#Create Test Set first from trimmed
print(df_fake_trim_2.shape)
trim_size = int(df_fake_trim_2.shape[0]*(1/3))
mask =  np.random.choice(df_fake_trim_2.shape[0], trim_size, replace=False) 
neg_mask = [i for i in range(0, df_fake_trim_2.shape[0]) if i not in mask]
df_fake_trim_2_test = df_fake_trim_2.iloc[mask]
df_fake_trim_2_train = df_fake_trim_2.iloc[neg_mask]
print(df_fake_trim_2_test.shape)
print(df_fake_trim_2_train.shape)

In [0]:
#combine both the real and trimmed fake news into one df
combined_train = pd.concat([df_real_train,df_fake_trim_2_train,df_grover_trim],ignore_index=True)
combined_test = pd.concat([df_real_test,df_fake_trim_2_test],ignore_index=True)

In [0]:
print(combined_train.shape)
print(combined_train.head())
combined_train.tail()

In [0]:
print(combined_test.shape)
print(combined_test.head())
combined_test.tail()

In [0]:
train_text = combined_train['clean_text'].values
test_text = combined_test['clean_text'].values
train_labels = combined_train['label'].values
test_labels = combined_test['label'].values

In [0]:
type(train_text)
print(train_text.shape)
print(test_text.shape)
fit_text = np.concatenate((train_text, test_text))
print(fit_text.shape)

In [0]:
tfidf_2 = TfidfVectorizer(ngram_range=(1,2), max_df= 0.85, min_df= 0.01)
tfidf_2.fit(fit_text)
train_text = tfidf_2.transform(train_text)
print(type(train_text))
print("tf idf vectorized text shape: {}".format(train_text.shape))
print("original text shape: {}".format(combined_train['clean_text'].shape))

test_text = tfidf_2.transform(test_text)
print(type(test_text))
print("tf idf vectorized text shape: {}".format(test_text.shape))
print("original text shape: {}".format(combined_test['clean_text'].shape))

In [0]:
nb_2 = MultinomialNB()
nb_2.fit(train_text,train_labels)
train_pred = nb_2.predict(train_text)

In [0]:
print('Naive Bayes In Sample F1 and Accuracy Scores:')
print('F1 score {:.4}%'.format(f1_score(train_labels, train_pred, average='macro')*100 ))
print ('Accuracy score {:.4}%'.format(accuracy_score(train_labels, train_pred)*100))

In [0]:
combined.loc[np.where(train_labels != train_pred)[0].tolist()].groupby('label').agg('count')

In [0]:
test_pred = nb_2.predict(test_text)
print('Naive Bayes Out of Sample F1 and Accuracy Scores:')
print('F1 score {:.4}%'.format(f1_score(test_labels, test_pred, average='macro')*100 ))
print ('Accuracy score {:.4}%'.format(accuracy_score(test_labels, test_pred)*100))

In [0]:
combined.loc[np.where(test_labels != test_pred)[0].tolist()].groupby('label').agg('count')